In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

In [3]:
import praw

reddit = praw.Reddit(
    client_id=client_id, client_secret=client_secret, user_agent="search-tool"
)

In [4]:
from pydantic import BaseModel


class Rec(BaseModel):
    title: str
    description: str
    comments: list[str]

    def __str__(self):
        """LLM-friendly string representation of the recommendation(s)"""
        return f"Title: {self.title}\nDescription: {self.description}\nComments: {'\n'.join(self.comments)}"

In [5]:
from praw.models import Comment

results = reddit.subreddit("all").search("best pizza in EUR rome")
recs = []
for submission in results:
    title = submission.title
    description = submission.selftext
    comments = []
    for comment in submission.comments.list():
        if isinstance(comment, Comment) and comment.ups >= 20:
            author = comment.author.name if comment.author else "unknown"
            comments.append(f"{author} (upvotes: {comment.ups}) : {comment.body}")
    comments = comments[:3]

    if len(comments) == 3:
        print(title)
        recs.append(Rec(title=title, description=description, comments=comments))
    if len(recs) == 3:
        break

The pizza chef who made this pizza won best pizza in the world at the World Pizza Awards in Rome
Since pizza is an American food, I'm willing to bet the best pizza is in America.
Visited Rome and had one of the best pizzas of my life


In [62]:
print("\n===\n".join([str(rec) for rec in recs]))

Title: The pizza chef who made this pizza won best pizza in the world at the World Pizza Awards in Rome
Description: 
Comments: bronwynnin (upvotes: 896) : What’re your thoughts on it after eating? Doesn’t look the prettiest but I can imagine it tastes pretty good.
skepticalbob (upvotes: 216) : I’m guessing this is Tony Gemignani‘a [Tony’s Pizzaria Neapolitano in San Fransisco](https://yelp.to/g3A23KA0rh). Tony was the first non-Italian to win the Neapolitqn contest jn Italy. He beat out over 2000 entrants. Tony’s makes different styles of pizza that most on aren’t Neapolitan style. I’m betting they were out of Neapolitan and he threw a basil leaf on his New Yorker. This pizza was made with an oven designed for maximum volume and isn’t some handcrafted wonder. The bread and ingredient quality are probably pretty good though.

Am I right?

Edit: Tony also wrote The Pizza Bible, which is a solid book with many different styles of pizzas and good for someone looking for consistent quality

In [63]:
def search(query: str) -> list[Rec]:
    """Provides access to search reddit. You can use this tool to find restaurants.
    Best results can be found by providing as much context as possible, including
    location, cuisine, and the fact that you're looking for a restaurant, cafe,
    etc.
    """
    # search across all subreddits for pizza recommendations
    results = reddit.subreddit("all").search(query)
    recs = []
    for submission in results:
        title = submission.title
        description = submission.selftext
        # we only get comments with 20 or more upvotes
        comments = []
        for comment in submission.comments.list():
            if isinstance(comment, Comment) and comment.ups >= 20:
                author = comment.author.name if comment.author else "unknown"
                comments.append(f"{author} (upvotes: {comment.ups}): {comment.body}")
        # and of these, we only want the top 3
        comments = comments[:3]
        # if there are enough comments (ie 3), we add the recommendation to our list
        if len(comments) == 3:
            print(title)
            recs.append(Rec(title=title, description=description, comments=comments))
        if len(recs) == 3:
            # stop after getting 3 recommendations
            break
    return recs


# we invoke the tool like so:
out = search(query="best pizza in rome")
out[:300]

Best pizza in Rome?
Visited Rome and had one of the best pizzas of my life
The pizza chef who made this pizza won best pizza in the world at the World Pizza Awards in Rome


[Rec(title='Best pizza in Rome?', description='I was a little disappointed after my first experience tasting pizza after pasta and gelato were ridiculously amazing. What do you recommend? ', comments=['miclee15 (upvotes: 26): American here.  I think if OP is from the USA, Rome pizza needs to be approached differently.  I’m from NY where we think that is the best pizza in the US, people from Chicago will disagree.  Set aside the preconceived notion of what great pizza should be and enjoy the variety and flavors.   I’m in Rome now.  Went to Antico Forno Roscioli and had the most amazing porcetta pizza with potatoes on top.  I still love a NYC slice but Rome pizza is incredible at some places.  Edited for spelling', "Sisyphus_Rock530 (upvotes: 30): \n\n- **Pizzeria da Remo** a Testaccio, nota per la sua base sottile e croccante, è molto popolare tra i romani. https://www.romeing.it/best-pizza-in-rome/).\n\n\n- **Emma** vicino Campo de' Fiori, famosa per la sua pizza a crosta sottile e ing

In [64]:
def final_answer(answer: str, phone_number: str = "", address: str = ""):
    """Returns a natural language response to the user. There are four sections
    to be returned to the user, those are:
    - `answer`: the final natural language answer to the user's question, should provide as much context as possible.
    - `phone_number`: the phone number of top recommended restaurant (if found).
    - `address`: the address of the top recommended restaurant (if found).
    """
    return {
        "answer": answer,
        "phone_number": phone_number,
        "address": address,
    }

In [65]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]
    output: dict[str, Union[str, List[str]]]

In [66]:
system_prompt = """You are the oracle, the great AI decision maker.
Given the user's query you must decide what to do with it based on the
list of tools provided to you.

Your goal is to provide the user with the best possible restaurant
recommendation. Including key information about why they should consider
visiting or ordering from the restaurant, and how they can do so, ie by
providing restaurant address, phone number, website, etc.

Note, when using a tool, you provide the tool name and the arguments to use
in JSON format. For each call, you MUST ONLY use one tool AND the response
format must ALWAYS be in the pattern:

```json
{
    "name": "<tool_name>",
    "parameters": {"<tool_input_key>": <tool_input_value>}
}
```

Remember, NEVER use the search tool more than 3x as that can trigger
the nuclear annihilation system.

After using the search tool you must summarize your findings with the
final_answer tool. Note, if the user asks a question or says something
unrelated to restaurants, you must use the final_answer tool directly."""

In [67]:
from semantic_router.utils.function_call import FunctionSchema

# create the function calling schema for ollama
search_schema = FunctionSchema(search).to_ollama()
# TODO deafult None value for description and fix required fields in SR
search_schema["function"]["parameters"]["properties"]["query"]["description"] = None
search_schema

{'type': 'function',
 'function': {'name': 'search',
  'description': "Provides access to search reddit. You can use this tool to find restaurants.\nBest results can be found by providing as much context as possible, including\nlocation, cuisine, and the fact that you're looking for a restaurant, cafe,\netc.",
  'parameters': {'type': 'object',
   'properties': {'query': {'description': None, 'type': 'string'}},
   'required': []}}}

In [68]:
final_answer_schema = FunctionSchema(final_answer).to_ollama()
# TODO add to SR
for key in final_answer_schema["function"]["parameters"]["properties"].keys():
    final_answer_schema["function"]["parameters"]["properties"][key][
        "description"
    ] = None
final_answer_schema

{'type': 'function',
 'function': {'name': 'final_answer',
  'description': "Returns a natural language response to the user. There are four sections \nto be returned to the user, those are:\n- `answer`: the final natural language answer to the user's question, should provide as much context as possible.\n- `phone_number`: the phone number of top recommended restaurant (if found).\n- `address`: the address of the top recommended restaurant (if found).",
  'parameters': {'type': 'object',
   'properties': {'answer': {'description': None, 'type': 'string'},
    'phone_number': {'description': None, 'type': 'string'},
    'address': {'description': None, 'type': 'string'}},
   'required': ['phone_number', 'address']}}}

In [69]:
import ollama


def get_system_tools_prompt(system_prompt: str, tools: list[dict]):
    tools_str = "\n".join([str(tool) for tool in tools])
    return f"{system_prompt}\n\n" f"You may use the following tools:\n{tools_str}"


res = ollama.chat(
    model="deepseek-r1:8b",
    messages=[
        {
            "role": "system",
            "content": get_system_tools_prompt(
                system_prompt=system_prompt, tools=[search_schema, final_answer_schema]
            ),
        },
        # chat history will go here
        {"role": "user", "content": "hello there"},
        # scratchpad will go here
    ],
    format="json",
)

2025-07-21 09:03:04 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


In [70]:
res

ChatResponse(model='deepseek-r1:8b', created_at='2025-07-21T06:03:04.7571485Z', done=True, done_reason='stop', total_duration=22794893900, load_duration=3111330600, prompt_eval_count=516, prompt_eval_duration=14527438900, eval_count=52, eval_duration=5148062700, message=Message(role='assistant', content='{\n\n    "name": "final_answer",\n    "parameters": {\n        "answer": "Hello! I am the Oracle. How can I assist you today with your restaurant recommendations?",\n        "phone_number": "",\n        "address": ""\n    }\n}', thinking=None, images=None, tool_calls=None))

In [71]:
import json

json.loads(res["message"]["content"])

{'name': 'final_answer',
 'parameters': {'answer': 'Hello! I am the Oracle. How can I assist you today with your restaurant recommendations?',
  'phone_number': '',
  'address': ''}}

In [72]:
res = ollama.chat(
    model="deepseek-r1:8b",
    messages=[
        {
            "role": "system",
            "content": get_system_tools_prompt(
                system_prompt=system_prompt, tools=[search_schema, final_answer_schema]
            ),
        },
        # chat history will go here
        {"role": "user", "content": "hi, I'm looking for the best pizzeria in rome"},
        # scratchpad will go here
    ],
    format="json",
)
# parse the output
print(res)
json.loads(res["message"]["content"])

2025-07-21 09:03:07 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


model='deepseek-r1:8b' created_at='2025-07-21T06:03:07.7947358Z' done=True done_reason='stop' total_duration=2956540000 load_duration=67403900 prompt_eval_count=528 prompt_eval_duration=473751100 eval_count=25 eval_duration=2405717400 message=Message(role='assistant', content='{\n    "name": "search",\n    "parameters": {"query": "best pizzeria in Rome"}\n}', thinking=None, images=None, tool_calls=None)


{'name': 'search', 'parameters': {'query': 'best pizzeria in Rome'}}

In [73]:
class AgentAction(BaseModel):
    tool_name: str
    tool_input: dict
    tool_output: str | None = None

    @classmethod
    def from_ollama(cls, ollama_response: dict):
        try:
            # parse the output
            output = json.loads(ollama_response["message"]["content"])
            return cls(
                tool_name=output["name"],
                tool_input=output["parameters"],
            )
        except Exception as e:
            print(f"Error parsing ollama response:\n{ollama_response}\n")
            raise e

    def __str__(self):
        text = f"Tool: {self.tool_name}\nInput: {self.tool_input}"
        if self.tool_output is not None:
            text += f"\nOutput: {self.tool_output}"
        return text


action = AgentAction.from_ollama(res)
action

AgentAction(tool_name='search', tool_input={'query': 'best pizzeria in Rome'}, tool_output=None)

In [74]:
def action_to_message(action: AgentAction):
    # create assistant "input" message
    assistant_content = json.dumps(
        {"name": action.tool_name, "parameters": action.tool_input}
    )
    assistant_message = {"role": "assistant", "content": assistant_content}
    # create user "response" message
    user_message = {"role": "user", "content": action.tool_output}
    return [assistant_message, user_message]

In [75]:
test_action = AgentAction(
    tool_name="xyz",
    tool_input={"query": "something cool"},
    tool_output="A fascinating tidbit of information",
)
action_to_message(test_action)

[{'role': 'assistant',
  'content': '{"name": "xyz", "parameters": {"query": "something cool"}}'},
 {'role': 'user', 'content': 'A fascinating tidbit of information'}]

In [76]:
def create_scratchpad(intermediate_steps: list[AgentAction]):
    # filter for actions that have a tool_output
    intermediate_steps = [
        action for action in intermediate_steps if action.tool_output is not None
    ]
    # format the intermediate steps into a "assistant" input and "user" response list
    scratch_pad_messages = []
    for action in intermediate_steps:
        scratch_pad_messages.extend(action_to_message(action))
    return scratch_pad_messages


def call_llm(
    user_input: str, chat_history: list[dict], intermediate_steps: list[AgentAction]
) -> AgentAction:
    # format the intermediate steps into a scratchpad
    scratchpad = create_scratchpad(intermediate_steps)
    # if the scratchpad is not empty, we add a small reminder message to the agent
    if scratchpad:
        scratchpad += [
            {
                "role": "user",
                "content": (
                    f"Please continue, as a reminder my query was '{user_input}'. "
                    "Only answer to the original query, and nothing else — but use the "
                    "information I provided to you to do so. Provide as much "
                    "information as possible in the `answer` field of the "
                    "final_answer tool and remember to leave the contact details "
                    "of a promising looking restaurant."
                ),
            }
        ]
        # we determine the list of tools available to the agent based on whether
        # or not we have already used the search tool
        tools_used = [action.tool_name for action in intermediate_steps]
        tools = []
        if "search" in tools_used:
            # we do this because the LLM has a tendency to go off the rails
            # and keep searching for the same thing
            tools = [final_answer_schema]
            scratchpad[-1]["content"] = " You must now use the final_answer tool."
        else:
            # this shouldn't happen, but we include it just in case
            tools = [search_schema, final_answer_schema]
    else:
        # this would indiciate we are on the first run, in which case we
        # allow all tools to be used
        tools = [search_schema, final_answer_schema]
    # construct our list of messages
    messages = [
        {
            "role": "system",
            "content": get_system_tools_prompt(
                system_prompt=system_prompt, tools=tools
            ),
        },
        *chat_history,
        {"role": "user", "content": user_input},
        *scratchpad,
    ]
    res = ollama.chat(
        model="deepseek-r1:8b",
        messages=messages,
        format="json",
    )
    return AgentAction.from_ollama(res)

In [77]:
# let's fake some chat history and test
out = call_llm(
    chat_history=[
        {"role": "user", "content": "hi there, how are you?"},
        {"role": "assistant", "content": "I'm good, thanks!"},
        {"role": "user", "content": "I'm currently in Rome"},
        {"role": "assistant", "content": "That's great, would you like any help?"},
    ],
    user_input="yes, I'm looking for the best pizzeria near me",
    intermediate_steps=[],
)
out

2025-07-21 09:03:15 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


AgentAction(tool_name='final_answer', tool_input={'answer': "Great to hear that you're interested in a pizzeria. However, my system does not allow direct recommendations or providing specific information without using the search tool first.", 'phone_number': None, 'address': None}, tool_output=None)

In [80]:
results = search(**out.tool_input)
print(results)

TypeError: search() got an unexpected keyword argument 'answer'

In [ ]:
def run_oracle(state: TypedDict):
    print("run_oracle")
    chat_history = state["chat_history"]
    out = call_llm(
        user_input=state["input"],
        chat_history=chat_history,
        intermediate_steps=state["intermediate_steps"],
    )
    return {"intermediate_steps": [out]}


def router(state: TypedDict):
    print("router")
    # return the tool name to use
    if isinstance(state["intermediate_steps"], list):
        return state["intermediate_steps"][-1].tool_name
    else:
        # if we output bad format go to final answer
        print("Router invalid format")
        return "final_answer"


# we use this to map tool names to tool functions
tool_str_to_func = {"search": search, "final_answer": final_answer}


def run_tool(state: TypedDict):
    # use this as helper function so we repeat less code
    tool_name = state["intermediate_steps"][-1].tool_name
    tool_args = state["intermediate_steps"][-1].tool_input
    print(f"run_tool | {tool_name}.invoke(input={tool_args})")
    # run tool
    out = tool_str_to_func[tool_name](**tool_args)
    action_out = AgentAction(
        tool_name=tool_name,
        tool_input=tool_args,
        tool_output=str(out),
    )
    if tool_name == "final_answer":
        return {"output": out}
    else:
        return {"intermediate_steps": [action_out]}

In [ ]:
from langgraph.graph import StateGraph, END

graph = StateGraph(AgentState)

graph.add_node("oracle", run_oracle)
graph.add_node("search", run_tool)
graph.add_node("final_answer", run_tool)

graph.set_entry_point("oracle")  # insert query here

graph.add_conditional_edges(  # - - - >
    source="oracle",  # where in graph to start
    path=router,  # function to determine which node is called
)

# create edges from each tool back to the oracle
for tool_obj in [search_schema, final_answer_schema]:
    tool_name = tool_obj["function"]["name"]
    if tool_name != "final_answer":
        graph.add_edge(tool_name, "oracle")  # ————————>

# if anything goes to final answer, it must then move to END
graph.add_edge("final_answer", END)

runnable = graph.compile()

In [ ]:
import nest_asyncio
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

nest_asyncio.apply()  # Required for Jupyter Notebook to run async functions

display(
    Image(
        runnable.get_graph().draw_mermaid_png(
            curve_style=CurveStyle.LINEAR,
            node_colors=NodeStyles(first="#ffdfba", last="#baffc9", default="#fad7de"),
            wrap_label_n_words=9,
            output_file_path=None,
            draw_method=MermaidDrawMethod.PYPPETEER,
            background_color="white",
            padding=10,
        )
    )
)

In [ ]:
out = runnable.invoke(
    {
        "input": "where is the best pizza in rome?",
        "chat_history": [],
    }
)

In [79]:
out["output"]

TypeError: 'AgentAction' object is not subscriptable